In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


device config


In [2]:
# check for CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [3]:
# load the cancer data set
data = load_breast_cancer()
X = data.data
y = data.target

In [4]:
# set up the test and training sets
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
print(xTrain.shape)
print(xTest.shape)

(455, 30)
(114, 30)


In [6]:
# standardize the data
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

In [7]:
# add the data to the cuda device
xTrain = torch.tensor(xTrain, dtype=torch.float32).to(device)
xTest = torch.tensor(xTest, dtype=torch.float32).to(device)
yTest = torch.tensor(yTest, dtype=torch.float32).to(device)
yTrain = torch.tensor(yTrain, dtype=torch.float32).to(device)

In [8]:
# create the neural network
class NeuralNetwork(nn.Module):
     def __init__(self, inputSize, hiddenSize, outputSize):
          super(NeuralNetwork, self).__init__()
          self.fc1 = nn.Linear(inputSize, hiddenSize)
          self.relu = nn.ReLU()
          self.fc2 = nn.Linear(hiddenSize, outputSize)
          self.sigmoid = nn.Sigmoid()

     def forward(self, x):
          out = self.fc1(x)
          out = self.relu(out)
          out = self.fc2(out)
          out = self.sigmoid(out)
          return out

In [9]:
# need to define the hyperparams
inputSize = xTrain.shape[1]
hiddenSize = 64
outputSize = 1
learningRate = 0.001
epochs = 100


In [10]:
# init the model and move it to GPU
model = NeuralNetwork(inputSize, hiddenSize, outputSize).to(device)

In [11]:
# define the loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), learningRate)

In [24]:
# train the model
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(xTrain)
  loss = criterion(outputs, yTrain.view(-1, 1))
  loss.backward()
  optimizer.step()
  # find the accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == yTrain.view(-1, 1)).float().sum()
    accuracy = correct / yTrain.size(0)
  if (epoch + 1) % 10 == 0:
     print(f"Epoch[{epoch + 1}/100]")
     print(f"Loss : {loss.item():.4f}")
     print(f"Accuracy {accuracy * 100}\n")

Epoch[10/100]
Loss : 0.0059
Accuracy 100.0

Epoch[20/100]
Loss : 0.0057
Accuracy 100.0

Epoch[30/100]
Loss : 0.0056
Accuracy 100.0

Epoch[40/100]
Loss : 0.0055
Accuracy 100.0

Epoch[50/100]
Loss : 0.0053
Accuracy 100.0

Epoch[60/100]
Loss : 0.0052
Accuracy 100.0

Epoch[70/100]
Loss : 0.0051
Accuracy 100.0

Epoch[80/100]
Loss : 0.0050
Accuracy 100.0

Epoch[90/100]
Loss : 0.0049
Accuracy 100.0

Epoch[100/100]
Loss : 0.0047
Accuracy 100.0



In [34]:
# model evaluation on training set
model.eval()
with torch.no_grad():
  outputs = model(xTrain)
  prediction = outputs.round()
  correction = (prediction == yTrain.view(-1, 1)).float().sum()
  accuracy = correction / yTrain.size(0)
  print(f"Accuracy on the training set is {accuracy * 100:.2f}")

Accuracy on the training set is 100.00


In [33]:
# model evaluation on test set
model.eval()
with torch.no_grad():
  outputs = model(xTest)
  prediction = outputs.round()
  correction = (prediction == yTest.view(-1, 1)).float().sum()
  accuracy = correction / yTest.size(0)
  print(f"Accuracy on the test set is {accuracy * 100:.2f}")

Accuracy on the test set is 97.37
